# Making the model and fitting it

In [1]:
#pip install datasets

In [2]:
import numpy as np
import tensorflow as tf
from datasets import load_dataset
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, Flatten, Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import img_to_array

np.random.seed(1)
tf.random.set_seed(1)

D:\Anaconda\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
def preprocess(dataset): 
    images = []
    labels = []
    for item in dataset:
        image_data = img_to_array(item['image'].convert('RGB'))
        image_data = tf.image.resize(image_data, (224, 224))
        images.append(image_data)
        labels.append(item['labels'])

    return np.array(images), np.array(labels)

# Load and preprocess the 'Bingsu/Cat_and_Dog' dataset
bingsu_cat_dog_dataset = load_dataset("Bingsu/Cat_and_Dog")
train_images_modified, train_labels_modified = preprocess(bingsu_cat_dog_dataset['train'])
test_images_modified, test_labels_modified = preprocess(bingsu_cat_dog_dataset['test'])

In [4]:
# Taking VGG16 model as our basement
vgg16_base = VGG16(weights='imagenet', include_top=False, input_tensor=Input(shape=(224, 224, 3)))
vgg16_base.trainable = False

# Modified model for catNdog
custom_flatten = Flatten()(vgg16_base.output)
custom_dense1 = Dense(512, activation='relu')(custom_flatten)
custom_output = Dense(1, activation='sigmoid')(custom_dense1)  
modified_model = Model(inputs=vgg16_base.input, outputs=custom_output)
modified_model.compile(optimizer=Adam(learning_rate=0.01), loss='binary_crossentropy', metrics=['accuracy'])

# Train 3 epochs not to overfit 
modified_model.fit(train_images_modified, train_labels_modified, batch_size=32, epochs=3, validation_split=0.2)

Epoch 1/3
200/200 [==============================] - 346s 2s/step - loss: 17.6023 - accuracy: 0.9391 - val_loss: 1.6682 - val_accuracy: 0.8519
Epoch 2/3
200/200 [==============================] - 352s 2s/step - loss: 0.3418 - accuracy: 0.9845 - val_loss: 0.5796 - val_accuracy: 0.9475
Epoch 3/3
200/200 [==============================] - 349s 2s/step - loss: 0.1164 - accuracy: 0.9911 - val_loss: 0.7728 - val_accuracy: 0.9362


# Final Result

In [5]:
test_accuracy = modified_model.evaluate(test_images_modified, test_labels_modified)[1]
print("The test accuracy is",test_accuracy)

63/63 [==============================] - 83s 1s/step - loss: 0.4490 - accuracy: 0.9675
The test accuracy is 0.9674999713897705
